In [1]:
import torch

print(torch.cuda.is_available())

True


In [17]:
import os

# [1] GCP 서비스 계정 키(JSON) 파일 경로
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\POLO\polo-system\models\math\stone-booking-466716-n6-f6fff7380e05.json"

# [2] GCP 프로젝트 ID  (구글 클라우드 콘솔 → 프로젝트 정보 → 프로젝트 ID)
os.environ["GOOGLE_CLOUD_PROJECT"] = "stone-booking-466716-n6"

# [3] 번역 API 지역 (기본은 global)
os.environ["GOOGLE_CLOUD_TRANSLATE_LOCATION"] = "global"

print("✅ 환경변수 설정 완료")


✅ 환경변수 설정 완료


In [18]:
import os, re, sys
from pathlib import Path
from typing import Dict, List, Tuple

from google.cloud import translate

# 환경변수 확인 (GOOGLE_APPLICATION_CREDENTIALS / GOOGLE_CLOUD_PROJECT / GOOGLE_CLOUD_TRANSLATE_LOCATION)
PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")
LOCATION   = os.getenv("GOOGLE_CLOUD_TRANSLATE_LOCATION", "global")  # 보통 'global'

if not PROJECT_ID:
    raise RuntimeError("환경변수 GOOGLE_CLOUD_PROJECT 가 설정되어 있지 않습니다.")
print(f"PROJECT_ID={PROJECT_ID}, LOCATION={LOCATION}")


PROJECT_ID=stone-booking-466716-n6, LOCATION=global


In [19]:
# 수식/LaTeX 블록은 번역에서 제외 → 보호 토큰으로 치환했다가 번역 후 원복합니다.
_MATH_ENV_NAMES = r"(?:equation|align|gather|multline|eqnarray|cases|split)\*?"
_MATH_PATTERN = re.compile(
    r"(?P<D2>\${2}[\s\S]*?\${2})"           # $$ ... $$
    r"|(?P<D1>(?<!\\)\$[\s\S]*?(?<!\\)\$)"  # $ ... $  (이스케이프된 \$는 제외)
    r"|(?P<LB>\\\[[\s\S]*?\\\])"            # \[ ... \]
    r"|(?P<LP>\\\([\s\S]*?\\\))"            # \( ... \)
    r"|(?P<ENV>\\begin\{"+_MATH_ENV_NAMES+r"\}[\s\S]*?\\end\{"+_MATH_ENV_NAMES+r"\})",
    re.MULTILINE
)

def protect_math(text: str) -> Tuple[str, Dict[str, str]]:
    holders: Dict[str, str] = {}
    def _repl(m):
        key = f"⟦MATH{len(holders)}⟧"
        holders[key] = m.group(0)
        return key
    return _MATH_PATTERN.sub(_repl, text), holders

def restore_math(text: str, holders: Dict[str, str]) -> str:
    for k, v in holders.items():
        text = text.replace(k, v)
    return text


In [20]:
# 요구사항: 괄호 안의 문자열은 번역하지 않습니다.
# 단일 레벨 괄호를 대상으로 하며 줄바꿈 포함 괄호는 제외합니다.
_PAREN_PATTERN = re.compile(r"\([^()\n]*\)")

def protect_parens(text: str) -> Tuple[str, Dict[str, str]]:
    holders: Dict[str, str] = {}
    idx = 0
    def _repl(m):
        nonlocal idx
        key = f"⦅PAREN{idx}⦆"
        holders[key] = m.group(0)
        idx += 1
        return key
    return _PAREN_PATTERN.sub(_repl, text), holders

def restore_parens(text: str, holders: Dict[str, str]) -> str:
    for k, v in holders.items():
        text = text.replace(k, v)
    return text


In [21]:
def split_into_paragraphs(s: str) -> List[str]:
    # 빈 줄 기준 문단 분리
    parts, cur = [], []
    for line in s.splitlines():
        if line.strip() == "":
            parts.append("\n".join(cur))
            parts.append("")  # 빈 문단 유지
            cur = []
        else:
            cur.append(line)
    parts.append("\n".join(cur))
    return parts

def join_paragraphs(paras: List[str]) -> str:
    # split 방식과 동일하게 빈 줄을 끼워넣어 원래 형태에 가깝게 복원
    out = []
    for i, p in enumerate(paras):
        out.append(p)
        if i < len(paras) - 1:
            out.append("")  # 빈 줄
    return "\n".join(out)


In [26]:
# Google Cloud Translation v3 클라이언트
client = translate.TranslationServiceClient()
PARENT = f"projects/{PROJECT_ID}/locations/{LOCATION}"
BATCH_SIZE = 32  # 너무 크게 잡으면 오류율↑

def translate_paragraphs_gcp(paragraphs: List[str], target_lang="ko") -> List[str]:
    """
    각 문단에 대해: 수식 보호 → 괄호 보호 → 번역 → 복원.
    - 빈 문단 또는 보호 후에도 실질적으로 빈 텍스트가 된 문단은 API 호출에서 제외(스킵)
    - 스킵한 문단은 원문 그대로(out_list에 빈 문자열 등)를 유지
    """
    # 1) 보호
    prot_list, holders_math, holders_paren = [], [], []
    for para in paragraphs:
        if not para.strip():
            # 완전 빈 문단: 보존만 하고 번역 스킵
            prot_list.append("")
            holders_math.append({})
            holders_paren.append({})
            continue
        p1, ph_m = protect_math(para)
        p2, ph_p = protect_parens(p1)
        prot_list.append(p2)
        holders_math.append(ph_m)
        holders_paren.append(ph_p)

    # 2) 배치 번역
    out_list = [""] * len(paragraphs)

    def _flush_batch(idxs: List[int]):
        if not idxs:
            return
        # 이 배치에서 **실제로 번역할 인덱스만** 추리기 (빈 내용 스킵)
        nonempty = [i for i in idxs if prot_list[i].strip() != ""]
        if not nonempty:
            # 전부 비어있으면 그대로 종료
            for i in idxs:
                out_list[i] = prot_list[i]  # 대부분 "", 즉 빈 문단
            return

        contents = [prot_list[i] for i in nonempty]
        try:
            resp = client.translate_text(
                request={
                    "parent": PARENT,
                    "contents": contents,
                    "mime_type": "text/plain",
                    "target_language_code": target_lang,
                }
            )
            translated = [t.translated_text for t in resp.translations]
        except Exception as e:
            print("[Translate Error][GCP]", e, file=sys.stderr)
            translated = contents  # 실패 시 원문 유지

        # 번역한 것만 복원/반영
        for j, idx in enumerate(nonempty):
            t = translated[j]
            t = restore_parens(t, holders_paren[idx])
            t = restore_math(t, holders_math[idx])
            out_list[idx] = t

        # 비어있는 항목은 원문(빈 문자열 등) 유지
        for i in set(idxs) - set(nonempty):
            out_list[i] = prot_list[i]

    buf = []
    for i in range(len(paragraphs)):
        buf.append(i)
        if len(buf) >= BATCH_SIZE:
            _flush_batch(buf); buf.clear()
    _flush_batch(buf)

    return out_list


In [27]:
def translate_tex_file(input_path: str, output_path: str | None = None) -> str:
    in_path = Path(input_path)
    if not in_path.exists():
        raise FileNotFoundError(f"입력 파일이 없습니다: {input_path}")

    tex = in_path.read_text(encoding="utf-8", errors="ignore")

    paras = split_into_paragraphs(tex)
    paras_tr = translate_paragraphs_gcp(paras, target_lang="ko")
    result = join_paragraphs(paras_tr)

    if output_path is None:
        out_path = in_path.with_suffix(".ko.tex") if in_path.suffix.lower() == ".tex" \
                   else Path(str(in_path) + ".ko.tex")
    else:
        out_path = Path(output_path)

    out_path.parent.mkdir(parents=True, exist_ok=True)
    out_path.write_text(result, encoding="utf-8")
    return str(out_path)


In [28]:
INPUT_TEX = r"C:\POLO\polo-system\models\math\_build_yolo\yolo_math_report.tex"  # ← 필요시 수정
OUTPUT_TEX = None  # None이면 yolo_math_report.ko.tex로 자동 저장
Path(INPUT_TEX).exists(), INPUT_TEX


(True,
 'C:\\POLO\\polo-system\\models\\math\\_build_yolo\\yolo_math_report.tex')

In [29]:
out_path = translate_tex_file(INPUT_TEX, OUTPUT_TEX)
print(f"[OK] 번역본 저장: {out_path}")


[OK] 번역본 저장: C:\POLO\polo-system\models\math\_build_yolo\yolo_math_report.ko.tex


In [30]:
# 번역된 파일의 앞부분 60줄만 미리보기
preview_lines = 60
with open(out_path, "r", encoding="utf-8", errors="ignore") as f:
    for i, line in enumerate(f):
        if i >= preview_lines: break
        print(line.rstrip("\n"))


\\documentclass[11pt]{article}
\\usepackage[margin=1in]{geometry}
\\usepackage{amsmath, amssymb, amsfonts}
\\usepackage{hyperref}
\\usepackage{kotex}
\\setlength{\\parskip}{6pt}
\\setlength{\\parindent}{0pt}
\\title{LaTeX 수식 설명 보고서 (Middle-School Level+)}
\\author{자동 파이프라인}
\\date{2025-09-11}
\\begin{document}
\\maketitle
\\tableofcontents
\\newpage



\\section*{문서 개요}
보조원
일반 기술 독자를 대상으로 명확하고 간결한 기술 문서를 작성합니다.



\새 페이지



\section*{91–91행 / inline(\$ \$) }
assistant
주어진 방정식을 단계별로 분석해 보겠습니다.



### 예시
방정식은 다음과 같습니다.
\[ 448 \times 448 \]



### 설명
이 방정식을 명확하게 설명하기 위해 방정식을 구성 요소와 그 역할로 나누어 보겠습니다.



1. ** 기호**: 이 기호는 곱셈을 나타냅니다. 448이라는 숫자에 그 숫자를 곱한다는 것을 나타냅니다.
2. ** 기호**: 이 기호는 448이라는 숫자를 나타냅니다. 448은 우리가 곱하는 숫자입니다.



따라서 \( 448 \times 448 \) 방정식은 448을 그 자체로 곱한다는 것을 의미합니다. 다시 말해, 448의 제곱을 구하는 것입니다.



### 결론
이 방정식의 핵심 목적은 숫자 448의 제곱을 계산하는 것입니다. 이 논문의 맥락에서, 이는 한 변의 길이가 448인 정사각형의 면적을 계산하는 등 다양한 목적에 유용할 수 있으며, 448의 제곱을 사용하여 어떤 양이나 값을 나타내는 더 큰 수학적 모델의 일부가 될 수도 있습니다.



따라서 결론은 다음과 같이 요약할 수 